# Probabilistic Bayesian Neural Networks

**Author:** [Khalid Salama](https://www.linkedin.com/in/khalid-salama-24403144/)<br>
**Date created:** 2021/01/15<br>
**Last modified:** 2021/01/15<br>
**Description:** Building probabilistic Bayesian neural network models with TensorFlow Probability.

## Introduction

Taking a probabilistic approach to deep learning allows to account for *uncertainty*,
so that models can assign less levels of confidence to incorrect predictions.
Sources of uncertainty can be found in the data, due to measurement error or
noise in the labels, or the model, due to insufficient data availability for
the model to learn effectively.


This example demonstrates how to build basic probabilistic Bayesian neural networks
to account for these two types of uncertainty.
We use [TensorFlow Probability](https://www.tensorflow.org/probability) library,
which is compatible with Keras API.

This example requires TensorFlow 2.3 or higher.
You can install Tensorflow Probability using the following command:

```python
pip install tensorflow-probability
```

## The dataset

We use the [Wine Quality](https://archive.ics.uci.edu/ml/datasets/wine+quality)
dataset, which is available in the [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/wine_quality).
We use the red wine subset, which contains 4,898 examples.
The dataset has 11numerical physicochemical features of the wine, and the task
is to predict the wine quality, which is a score between 0 and 10.
In this example, we treat this as a regression task.

You can install TensorFlow Datasets using the following command:

```python
pip install tensorflow-datasets
```

## Setup

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

## Create training and evaluation datasets

Here, we load the `wine_quality` dataset using `tfds.load()`, and we convert
the target feature to float. Then, we shuffle the dataset and split it into
training and test sets. We take the first `train_size` examples as the train
split, and the rest as the test split.

In [7]:

def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", data_dir="/p/project/training2305/course-material-upd/BLcourse4/datasets/", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset


## Compile, train, and evaluate the model

In [8]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")


## Create model inputs

In [9]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs


## Experiment 1: standard neural network

We create a standard deterministic neural network model as a baseline.

In [10]:

def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


Let's split the wine dataset into training and test sets, with 85% and 15% of
the examples, respectively.

In [11]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-03-24 00:48:26.870482: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14628 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:01:00.0, compute capability: 7.0
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Now let's train the baseline model. We use the `MeanSquaredError`
as the loss function.

In [12]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100


2023-03-24 00:48:44.574518: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-03-24 00:48:44.576634: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-03-24 00:48:45.665693: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x1d48f450 initialized for platform CUDA (this does not guaran

17/17 [==============================] - 3s 29ms/step - loss: 28.1717 - root_mean_squared_error: 5.3077 - val_loss: 30.2676 - val_root_mean_squared_error: 5.5016
Epoch 2/100
17/17 [==============================] - 0s 10ms/step - loss: 26.5429 - root_mean_squared_error: 5.1520 - val_loss: 28.8875 - val_root_mean_squared_error: 5.3747
Epoch 3/100
17/17 [==============================] - 0s 8ms/step - loss: 25.1419 - root_mean_squared_error: 5.0142 - val_loss: 27.3569 - val_root_mean_squared_error: 5.2304
Epoch 4/100
17/17 [==============================] - 0s 9ms/step - loss: 23.7807 - root_mean_squared_error: 4.8765 - val_loss: 25.7095 - val_root_mean_squared_error: 5.0705
Epoch 5/100
17/17 [==============================] - 0s 9ms/step - loss: 22.4434 - root_mean_squared_error: 4.7374 - val_loss: 23.9920 - val_root_mean_squared_error: 4.8982
Epoch 6/100
17/17 [==============================] - 0s 9ms/step - loss: 21.1341 - root_mean_squared_error: 4.5972 - val_loss: 22.2725 - val_root

We take a sample from the test set use the model to obtain predictions for them.
Note that since the baseline model is deterministic, we get a single a
*point estimate* prediction for each test example, with no information about the
uncertainty of the model nor the prediction.

In [13]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 5.7 - Actual: 6.0
Predicted: 6.2 - Actual: 7.0
Predicted: 6.0 - Actual: 7.0
Predicted: 6.5 - Actual: 6.0
Predicted: 6.4 - Actual: 5.0
Predicted: 6.5 - Actual: 7.0
Predicted: 6.0 - Actual: 6.0
Predicted: 6.0 - Actual: 6.0
Predicted: 6.6 - Actual: 8.0
Predicted: 5.6 - Actual: 6.0


## Experiment 2: Bayesian neural network (BNN)

The object of the Bayesian approach for modeling neural networks is to capture
the *epistemic uncertainty*, which is uncertainty about the model fitness,
due to limited training data.

The idea is that, instead of learning specific weight (and bias) *values* in the
neural network, the Bayesian approach learns weight *distributions*
- from which we can sample to produce an output for a given input -
to encode weight uncertainty.

Thus, we need to define prior and the posterior distributions of these weights,
and the training process is to learn the parameters of these distributions.

In [14]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model


We use the `tfp.layers.DenseVariational` layer instead of the standard
`keras.layers.Dense` layer in the neural network model.

In [15]:

def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


The epistemic uncertainty can be reduced as we increase the size of the
training data. That is, the more data the BNN model sees, the more it is certain
about its estimates for the weights (distribution parameters).
Let's test this behaviour by training the BNN model on a small subset of
the training set, and then on the full training set, to compare the output variances.

### Train BNN  with a small training subset.

In [16]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Start training the model...
Epoch 1/500
5/5 [==============================] - 4s 557ms/step - loss: 57.0129 - root_mean_squared_error: 7.5497 - val_loss: 56.7286 - val_root_mean_squared_error: 7.5309
Epoch 2/500
5/5 [==============================] - 0s 29ms/step - loss: 52.7787 - root_mean_squared_error: 7.2640 - val_loss: 49.4205 - val_root_mean_squared_error: 7.0291
Epoch 3/500
5/5 [==============================] - 0s 28ms/step - loss: 55.1672 - root_mean_squared_error: 7.4266 - val_loss: 53.8305 - val_root_mean_squared_error: 7.3360
Epoch 4/500
5/5 [==============================] - 0s 30ms/step - loss: 54.0287 - root_mean_squared_error: 7.3497 - val_loss: 50.6124 - val_root_mean_squared_error: 7.1131
Epoch 5/500
5/5 [==============================] - 0s 30ms/step - loss: 50.9889 - root_mean_squared_error: 7.1395 - val_loss: 51.2843 - val_root_mean_squared_error: 7.1604
Epoch 6/500
5/5 [==============================] - 0s 30ms/step - loss: 51.8222 - root_mean_squared_error: 7.19

Since we have trained a BNN model, the model produces a different output each time
we call it with the same input, since each time a new set of weights are sampled
from the distributions to construct the network and produce an output.
The less certain the mode weights are, the more variability (wider range) we will
see in the outputs of the same inputs.

In [17]:

def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 5.74, min: 5.14, max: 6.27, range: 1.13 - Actual: 6.0
Predictions mean: 6.1, min: 5.56, max: 6.33, range: 0.77 - Actual: 7.0
Predictions mean: 5.84, min: 5.34, max: 6.34, range: 1.0 - Actual: 7.0
Predictions mean: 6.15, min: 5.61, max: 6.39, range: 0.78 - Actual: 6.0
Predictions mean: 6.11, min: 5.58, max: 6.37, range: 0.79 - Actual: 5.0
Predictions mean: 6.01, min: 5.37, max: 6.36, range: 0.99 - Actual: 7.0
Predictions mean: 6.04, min: 5.51, max: 6.41, range: 0.91 - Actual: 6.0
Predictions mean: 5.96, min: 5.42, max: 6.32, range: 0.9 - Actual: 6.0
Predictions mean: 6.14, min: 5.65, max: 6.43, range: 0.78 - Actual: 8.0
Predictions mean: 5.66, min: 4.99, max: 6.2, range: 1.21 - Actual: 6.0


### Train BNN  with the whole training set.

In [18]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 2s 35ms/step - loss: 40.0905 - root_mean_squared_error: 6.3314 - val_loss: 35.8348 - val_root_mean_squared_error: 5.9858
Epoch 2/500
17/17 [==============================] - 0s 10ms/step - loss: 37.6488 - root_mean_squared_error: 6.1355 - val_loss: 37.0272 - val_root_mean_squared_error: 6.0846
Epoch 3/500
17/17 [==============================] - 0s 11ms/step - loss: 36.3662 - root_mean_squared_error: 6.0301 - val_loss: 37.7250 - val_root_mean_squared_error: 6.1417
Epoch 4/500
17/17 [==============================] - 0s 10ms/step - loss: 34.3783 - root_mean_squared_error: 5.8630 - val_loss: 31.5982 - val_root_mean_squared_error: 5.6208
Epoch 5/500
17/17 [==============================] - 0s 10ms/step - loss: 35.5805 - root_mean_squared_error: 5.9646 - val_loss: 30.3964 - val_root_mean_squared_error: 5.5129
Epoch 6/500
17/17 [==============================] - 0s 10ms/step - loss: 32.8945 - root_mean_squared_

Notice that the model trained with the full training dataset shows smaller range
(uncertainty) in the prediction values for the same inputs, compared to the model
trained with a subset of the training dataset.

## Experiment 3: probabilistic Bayesian neural network

So far, the output of the standard and the Bayesian NN models that we built is
deterministic, that is, produces a point estimate as a prediction for a given example.
We can create a probabilistic NN by letting the model output a distribution.
In this case, the model captures the *aleatoric uncertainty* as well,
which is due to irreducible noise in the data, or to the stochastic nature of the
process generating the data.

In this example, we model the output as a `IndependentNormal` distribution,
with learnable mean and variance parameters. If the task was classification,
we would have used `IndependentBernoulli` with binary classes, and `OneHotCategorical`
with multiple classes, to model distribution of the model output.

In [19]:

def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


Since the output of the model is a distribution, rather than a point estimate,
we use the [negative loglikelihood](https://en.wikipedia.org/wiki/Likelihood_function)
as our loss function to compute how likely to see the true data (targets) from the
estimated distribution produced by the model.

In [20]:

def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 3s 55ms/step - loss: 49.4791 - root_mean_squared_error: 6.0334 - val_loss: 47.2343 - val_root_mean_squared_error: 6.0839
Epoch 2/1000
17/17 [==============================] - 0s 10ms/step - loss: 43.4683 - root_mean_squared_error: 5.8431 - val_loss: 41.7599 - val_root_mean_squared_error: 5.6779
Epoch 3/1000
17/17 [==============================] - 0s 11ms/step - loss: 40.1974 - root_mean_squared_error: 5.8785 - val_loss: 26.8484 - val_root_mean_squared_error: 5.5150
Epoch 4/1000
17/17 [==============================] - 0s 10ms/step - loss: 26.8967 - root_mean_squared_error: 5.8974 - val_loss: 19.8879 - val_root_mean_squared_error: 5.5704
Epoch 5/1000
17/17 [==============================] - 0s 10ms/step - loss: 26.5412 - root_mean_squared_error: 5.6827 - val_loss: 20.3914 - val_root_mean_squared_error: 5.5372
Epoch 6/1000
17/17 [==============================] - 0s 10ms/step - loss: 27.7963 - root_mean_sq

Now let's produce an output from the model given the test examples.
The output is now a distribution, and we can use its mean and variance
to compute the confidence intervals (CI) of the prediction.

In [25]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

AttributeError: 'Independent' object has no attribute 'sddev'